<a href="https://colab.research.google.com/github/ItsmeJ0/CourseUdemyAI/blob/main/Project_Advanced_Recommender_Systems_Movie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
pip install scikit-learn


In [19]:
import numpy as np
import pandas as pd
column_names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('u.data', sep='\t', names=column_names)
df.head()

,user_id,item_id,rating,timestamp
0,0,50,5,881250949
1,0,172,5,881250949
2,0,133,1,881250949
3,196,242,3,881250949
4,186,302,3,891717742


In [20]:
movie_titles = pd.read_csv("Movie_Id_Titles")
movie_titles.head()

,item_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [21]:
df = pd.merge(df,movie_titles,on='item_id')
df.head()

,user_id,item_id,rating,timestamp,title
0,0,50,5,881250949,Star Wars (1977)
1,290,50,5,880473582,Star Wars (1977)
2,79,50,4,891271545,Star Wars (1977)
3,2,50,5,888552084,Star Wars (1977)
4,8,50,5,879362124,Star Wars (1977)


In [22]:
# Menghitung jumlah pengguna unik (users) dalam dataframe df
n_users = df['user_id'].nunique()

# Menghitung jumlah film unik (movies) dalam dataframe df
n_items = df['item_id'].nunique()

# Mencetak hasil jumlah pengguna dan jumlah film
print('Num. of Users: ' + str(n_users))
print('Num of Movies: ' + str(n_items))


Num. of Users: 944
Num of Movies: 1682


In [23]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(df, test_size=0.25)


In [24]:
# Inisialisasi matriks untuk data latihan (train data)
train_data_matrix = np.zeros((n_users, n_items))

# Loop melalui data latihan dan mengisi matriks
for line in train_data.itertuples():
    # line[1] adalah user_id, line[2] adalah item_id, line[3] adalah rating
    train_data_matrix[line[1]-1, line[2]-1] = line[3]

# Inisialisasi matriks untuk data uji (test data)
test_data_matrix = np.zeros((n_users, n_items))

# Loop melalui data uji dan mengisi matriks
for line in test_data.itertuples():
    # line[1] adalah user_id, line[2] adalah item_id, line[3] adalah rating
    test_data_matrix[line[1]-1, line[2]-1] = line[3]


In [25]:
from sklearn.metrics.pairwise import pairwise_distances
user_similarity = pairwise_distances(train_data_matrix, metric='cosine')
item_similarity = pairwise_distances(train_data_matrix.T, metric='cosine')

In [29]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        # Menghitung rata-rata peringkat pengguna
        mean_user_rating = ratings.mean(axis=1)

        # Menghitung selisih peringkat dari rata-rata peringkat pengguna
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])

        # Melakukan prediksi peringkat berdasarkan kesamaan dan selisih peringkat
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        # Melakukan prediksi peringkat berdasarkan matriks peringkat dan kesamaan item
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])

    return pred


In [30]:
# Membuat prediksi peringkat berdasarkan kesamaan item (item-based recommendation)
item_prediction = predict(train_data_matrix, item_similarity, type='item')

# Membuat prediksi peringkat berdasarkan kesamaan pengguna (user-based recommendation)
user_prediction = predict(train_data_matrix, user_similarity, type='user')


In [34]:
from sklearn.metrics import mean_squared_error
from math import sqrt

def rmse(prediction, ground_truth):
    # Mengambil hanya elemen-elemen yang tidak nol (nonzero) dalam prediksi dan data aktual
    prediction = prediction[ground_truth.nonzero()].flatten()
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()

    # Menghitung Mean Squared Error (MSE) antara prediksi dan data aktual
    mse = mean_squared_error(prediction, ground_truth)

    # Menghitung RMSE sebagai akar kuadrat dari MSE
    return sqrt(mse)
print('User-based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix)))
print('Item-based CF RMSE: ' + str(rmse(item_prediction, test_data_matrix)))


User-based CF RMSE: 3.1286592446100605
Item-based CF RMSE: 3.4544701704741985


In [35]:
sparsity=round(1.0-len(df)/float(n_users*n_items),3)
print('The sparsity level of MovieLens100K is ' +  str(sparsity*100) + '%')

The sparsity level of MovieLens100K is 93.7%


In [36]:
import scipy.sparse as sp
from scipy.sparse.linalg import svds

# Mendapatkan komponen SVD dari matriks data latihan. Pilih jumlah dimensi k.
u, s, vt = svds(train_data_matrix, k=20)

# Membuat matriks diagonal dari singular values (s)
s_diag_matrix = np.diag(s)

# Mendapatkan prediksi matriks dengan mengalikan matriks SVD yang telah diperoleh
# (u, s_diag_matrix, vt)
X_pred = np.dot(np.dot(u, s_diag_matrix), vt)

# Menghitung RMSE untuk model rekomendasi berdasarkan user
print('User-based CF MSE: ' + str(rmse(X_pred, test_data_matrix)))


User-based CF MSE: 2.722921550263238
